In [2]:
#!/usr/bin/env python

# Copyright (c) 2021- The University of Notre Dame.
# This software is distributed under the GNU General Public License.
# See the file COPYING for details.

# This program is a demonstration of using Work Queue with PyTask
# PyTask creates a Work Queue task using a python function and its argument and returns the output
# This example program finds prime numbers in a range
# Finding the primality of each number is created as a seperate task
# The tasks for all numbers in the given range are sent out to workers and then computed and returned

# How to run this program:
# Run this cell, and afterwards two sliders will appear which will allow you to adjust the number range and number of Work Queue workers
# It is recommended that the default values are left for the first run through
# Afterwards, run the cell below to create the Work Queue queue, the workers, and execute the tasks
# Real time output, as well as some informative diagnostic graphs, will be shown at the very bottom of the notebook
# Grey cells are tasks that have been submitted, yellow cells are tasks that are currently being worked on,
# Green tasks are completed tasks returned by workers. Light green represents that the number is prime, dark green is composite
# After the Workqueue Status textbox displays "All tasks complete!", the program is finished! 
# Then feel free to play around with the number range and number of worker variables

import work_queue as wq
import ipywidgets as widgets
import time
import widget_control 
import os

display = widget_control.Display() # display class to handle all the fancy output widgets
display.create_user_input_widgets() # create the user input widgets below this cell

In [ ]:
# function to create the widget display at the bottom of the notebook
display.create_output_widgets()

# The function below takes in a number as an input and returns 0 if the number is composite or 1 if the number is prime
# PyTask uses this function to create the tasks given to workers to determine the primality of numbers in a range
def is_prime(number): 
    import math # PyTask requires that all libraries used in the function are included inside the function
    for i in range(2, int(math.sqrt(number)) + 1):
            if (number % i == 0):
                return 0 # if the number has a divisor, it is composite
    return 1 # otherwise it is prime

# create the Work Queue queue and specify the number of workers as well as the properties of those workers
q = wq.WorkQueue(port=[9120, 9129]) # specify the port to create the workers on. 0 selects a random unused port
q.specify_name("octo")
print(q.port)

# the line below is a fault tolerant method of instantiating the Work Queue workers as well as cleaning them up when finished
# loop through the range of numbers desired and use PyTask to create tasks to determine their primality
for i in range(display.tasks_range.value[0], display.tasks_range.value[1] + 1):
    p_task = wq.PythonTask(is_prime, i) # create the task using PyTask, supplying the function and its argument(s)
    p_task.specify_environment("worker-env.tar.gz") # Because the function uses the math library and PyTask requires the dill library, we must specify an enviroment for the workers that contains these elements
    q.submit(p_task) # submit the tasks into the queue

while not q.empty(): # continue until all the tasks have been completed
    t = q.wait(5) # give control to Work Queue so that it can talk to workers
    if t:
        if t.return_status != 0: # The task failed
            continue
        display.update_output_widgets(q, t) # update output widgets 
                              